In [1]:
%cd ../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [2]:
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types, ALL_LITERALS
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import datetime
from typing import List, Sequence, Tuple
from utils import URI_PREFIX
from preprocess.binning import delete_empty_bin_types

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataload import dmg777k
data = dmg777k()


loaded data dmg777k (65.61s).
pruned (12.03s).


In [28]:
from pykeen import models
from torch.optim import Adam
from pykeen.training import SLCWATrainingLoop, LCWATrainingLoop, TrainingLoop
import pandas as pd
from pykeen.triples import TriplesFactory
from typing import List, Sequence, Tuple
from pyrdf2vec.typings import Embeddings
from pykeen.utils import resolve_device
from utils.data_utils import data_to_kg, extract_ents
import os
FILEPATH = "data/preprocessed"
def data_to_pykeen(data):
    if not os.path.exists(f'{FILEPATH}/{data.name}.tsv.gz'):
        print('pykeen file does not exist. Writing pykeen file...')
        print(data.name)
        df =pd.DataFrame(data.triples, columns=["h","r","t"])
        df['h'] = df['h'].apply(lambda h:data.i2e[h][0])
        df['r'] = df['r'].apply(lambda r:data.i2r[r])
        df['t'] = df['t'].apply(lambda t:data.i2e[t][0])
        df.to_csv(f'{FILEPATH}/{data.name}.tsv.gz', 
                index=False,
                sep="\t" ,
                compression="gzip")
    return  TriplesFactory.from_path(f'{FILEPATH}/{data.name}.tsv.gz')
        

In [4]:
from utils.data_utils import data_to_kg, extract_ents, update_dataset_name, ensure_data_symmetry
data = ensure_data_symmetry(data)

In [5]:
from preprocess import delete_all_literals
data = delete_all_literals(data)


In [ ]:
fr

In [55]:
model = models.RGCN(triples_factory=training_triples_factory,
            embedding_dim=models.RGCN.hpo_default['embedding_dim']['low'],
            num_layers=models.RGCN.hpo_default['num_layers']['low'],

activation=models.RGCN.hpo_default['activation_cls']['choices'][0],
interaction=models.RGCN.hpo_default['interaction']['choices'][0],
edge_dropout=models.RGCN.hpo_default['edge_dropout']['low'],
self_loop_dropout=models.RGCN.hpo_default['self_loop_dropout']['low'],
edge_weighting=models.RGCN.hpo_default['edge_weighting']['choices'][0],
decomposition=models.RGCN.hpo_default['decomposition']['choices'][0]
            )

No random seed is specified. This may lead to non-reproducible results.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(7, 1024)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(7, 1024)
      )
    )
  )
  (self_loop): Linear(in_features=32, out_features=32, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
) has parameters, but no reset_parameters.


In [48]:
models.RGCN.hpo_default['decomposition']

{'type': 'categorical', 'choices': ['bases', 'blocks']}

In [35]:
models.RGCN.hpo_default['embedding_dim']['low']

32

In [32]:
training_triples_factory = data_to_pykeen(data)
models.RGCN.hpo_default['embedding_dim']
model = models.RGCN(triples_factory=training_triples_factory, 
                base_entity_initializer= "xavier_uniform" ,decomposition="bases",
                   decomposition_kwargs= {"num_bases": 2} ,    edge_dropout= 0.4,
    edge_weighting= "InverseInDegree",
    embedding_dim= 200,
    interaction= "distmult",
    num_layers= 1,
    regularizer= "Lp",
    regularizer_kwargs= {
      "weight": 0.01},
    relation_initializer= "xavier_uniform",
    self_loop_dropout= 0.2, 

                     )
model = model.to(resolve_device('gpu'))

optimizer = Adam(params=model.get_grad_params())
training_loop = LCWATrainingLoop( model=model,
        triples_factory=training_triples_factory,
        optimizer=optimizer,
    )


No random seed is specified. This may lead to non-reproducible results.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(2, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(2, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.


In [52]:
training_loop.train(
triples_factory=training_triples_factory,batch_size=30000)
embeddings = model.entity_representations[0]().detach().cpu().numpy()
reorder = []
for e in data.i2e:
    reorder.append(training_triples_factory.entity_to_id[e[0]])
embeddings = embeddings[reorder]
train_entities, test_entities, train_target, test_taget = extract_ents(
        data)  # extract necessary fields from data
train_embeddings = embeddings[data.training[:,0]]
test_embeddings = embeddings[data.withheld[:,0]]


Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(2, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(2, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.


KeyboardInterrupt: 

In [11]:
from embed import ComplEx2
embedder = ComplEx2(data)


No random seed is specified. This may lead to non-reproducible results.


In [8]:
train_entities, test_entities, train_target, test_taget = extract_ents(
    data)  

In [12]:
embeddings, train_embeddings, test_embeddings = embedder.fit_transform()

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\pykeen\triples\triples_factory.py:543: DeprecationWarning: Training instances are always shuffled.
  warnings.warn("Training instances are always shuffled.", DeprecationWarning)
Training epochs on cuda:0:   0%|          | 0/5 [00:00<?, ?epoch/s]c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\pykeen\triples\triples_factory.py:543: DeprecationWarning: Training instances are always shuffled.
  warnings.warn("Training instances are always shuffled.", DeprecationWarning)

Training epochs on cuda:0: 100%|██████████| 5/5 [06:09<00:00, 73.81s/epoch, loss=1.25, prev_loss=1.49]


In [24]:
embeddings.dtype == "complex64"

True

In [27]:
train_embeddings.real

array([[-0.9777068 , -0.7635542 ,  0.83458006, ...,  0.41527963,
        -0.22074403, -0.8594342 ],
       [-0.2137555 , -0.8035449 , -0.7950102 , ..., -0.44129416,
        -0.14687946, -0.8657731 ],
       [-0.73207545,  0.17865662,  1.0127183 , ..., -0.4405108 ,
         1.5906857 , -1.0679837 ],
       ...,
       [ 0.7201395 , -1.0935488 , -0.10072973, ..., -0.25403646,
        -1.584759  ,  0.3201824 ],
       [-0.4441676 , -1.9262924 ,  0.23936209, ..., -0.2884364 ,
         1.3789079 , -0.5678891 ],
       [-0.7182511 ,  0.40814206,  0.13491729, ..., -0.54138774,
         0.31239837, -0.97729874]], dtype=float32)

In [26]:
train_embeddings

array([[-0.9777068 -0.6177472j , -0.7635542 -0.16309066j,
         0.83458006-1.1783129j , ...,  0.41527963-1.1748574j ,
        -0.22074403+1.2493229j , -0.8594342 +0.03870127j],
       [-0.2137555 -1.862195j  , -0.8035449 -0.33094424j,
        -0.7950102 +0.61856586j, ..., -0.44129416+0.6823154j ,
        -0.14687946+0.34228072j, -0.8657731 -0.39387408j],
       [-0.73207545+0.8088891j ,  0.17865662+0.9209582j ,
         1.0127183 +0.7342377j , ..., -0.4405108 +0.03810455j,
         1.5906857 +0.5043095j , -1.0679837 +0.30315158j],
       ...,
       [ 0.7201395 -0.10735061j, -1.0935488 +1.5843045j ,
        -0.10072973-1.6161357j , ..., -0.25403646+1.1185342j ,
        -1.584759  +0.9799611j ,  0.3201824 +0.02990701j],
       [-0.4441676 -0.40123677j, -1.9262924 +1.9963207j ,
         0.23936209+2.4795065j , ..., -0.2884364 +0.13580966j,
         1.3789079 +0.69257826j, -0.5678891 -0.66061157j],
       [-0.7182511 +0.83655214j,  0.40814206-1.2147427j ,
         0.13491729+1.2358903j

In [13]:
if embeddings.dtype == "complex64":
    train_embeddings = train_embeddings.real
    test_embeddings = test_embeddings.real
    
    

In [14]:
from evaluate import KNN
model = KNN()
model.fit(train_embeddings, train_target)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [10, 20, 30],
                         'n_neighbors': [3, 5, 8, 15]})

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
predictions = model.predict(test_embeddings)
print(
    f"Predicted {len(test_entities)} entities with an accuracy of "
    + f"{accuracy_score(test_taget, predictions) * 100 :.4f}%"
    )
print(f'resulted in following f scores: micro {f1_score(test_taget, predictions, average="micro")} macro {f1_score(test_taget, predictions, average="macro")}')
print("Confusion Matrix :")
print(confusion_matrix(test_taget, predictions))

Predicted 2001 entities with an accuracy of 43.4783%
resulted in following f scores: micro 0.43478260869565216 macro 0.17127417827920058
Confusion Matrix :
[[  0  64   0  11   1]
 [  1 763   1 174  17]
 [  0 119   0  24   1]
 [  0 442   0 101  12]
 [  0 206   0  58   6]]


In [ ]:

# TODO pack into 1 step or create map instead of list to be able to know model name.
models = {}
for m in cfg["pipeline"]["evaluate"]:
    print(f'fitting {m}...')
    model = getattr(evaluate, m)(
        **cfg["evaluate"][m])
    model.fit(train_embeddings, train_target)
    models[m] = model

print("Evaluation started...")
# TODO save into file to analyze in subsequent stages (also with _0,_1 and so on)
    # prio 1.1
for m, model in models.items():
    print(f"evaluating model {m}")
    predictions = model.predict(test_embeddings)
    version = 0
    predictions_base_path = f'{cfg["file_paths"]["predicted"]}/{data.name}${cfg["pipeline"]["embed"]}${m}'
    while(os.path.exists(f'{predictions_base_path}${str(version)}.csv')):
        version +=1
    np.savetxt(f'{predictions_base_path}${str(version)}.csv',[predictions, test_taget],delimiter=',',fmt="%s")

    print(
        f"Predicted {len(test_entities)} entities with an accuracy of "
        + f"{accuracy_score(test_taget, predictions) * 100 :.4f}%"
        )
    print(f'resulted in following f scores: micro {f1_score(test_taget, predictions, average="micro")} macro {f1_score(test_taget, predictions, average="macro")}')
    print("Confusion Matrix :")
    print(confusion_matrix(test_taget, predictions))



print("Save Data...")